In [68]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

In [69]:
#PCSI0023

In [70]:
#Read the data

GenomicFeatures_PCSI0023 = pd.read_csv(open('data/PCSI0023/GenomicFeatures_PCSI0023.txt','r'),sep='\t')
SNVCalls_PCSI0023    = pd.read_csv(open('data/PCSI0023/SNVCalls_PCSI0023.txt','r'),sep='\t') 

In [71]:
print GenomicFeatures_PCSI0023.shape
print SNVCalls_PCSI0023.shape

(93177, 15)
(93177, 133)


In [72]:
np.sum(GenomicFeatures_PCSI0023.isnull())

CHROM                  0
POS                    0
RefAllele              0
NonRefAllele           0
BaseQual               0
TumourCoverage         0
NormalCoverage         0
MapQual                0
ReadPosition         332
Trinucleotide          0
HomopolymerRate        0
GCcontent              0
GenomicLocation    34951
DistanceSNP            0
StrandBias           332
dtype: int64

In [73]:
GenomicFeatures_PCSI0023['ReadPosition'].fillna(0, inplace=True)  #Replace na with 0
GenomicFeatures_PCSI0023['StrandBias'].fillna(0, inplace=True)

#print np.sum(GenomicFeatures_PCSI0023.isnull())

In [74]:
np.unique(GenomicFeatures_PCSI0023['GenomicLocation'])

array(['UTR3', 'UTR5', 'downstream', 'exonic', 'exonic;splicing',
       'intergenic', 'intronic', 'nan', 'ncRNA_UTR3', 'ncRNA_UTR5',
       'ncRNA_exonic', 'ncRNA_intronic', 'splicing', 'upstream',
       'upstream;downstream'], 
      dtype='|S19')

In [75]:
GenomicFeatures_PCSI0023.head()

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,16014,138,47,5607,193,103,0,57.0,ACT,3.201550,0.656716,ncRNA_intronic,67070,0.523316
1,chr1,16534,69,46,3187,128,92,10,65.5,ACA,2.615385,0.442786,ncRNA_intronic,66550,0.078125
2,chr1,16949,194,34,6167,242,136,0,37.5,ATT,2.388889,0.597015,ncRNA_exonic,66135,0.921488
3,chr1,17020,249,44,8271,305,168,0,56.0,GCC,2.437500,0.587065,NaN,66064,0.881967
4,chr1,17746,326,55,11417,386,245,0,51.0,CTC,2.570423,0.587065,ncRNA_intronic,65338,0.274611


In [76]:
SNVCalls_PCSI0023.head()

,CHROM,POS,END,X2399007,X2701027,X2701030,X2762077,X2762079,X2811068,X2811070,X2811073,X2811076,X2811391,X2812908,X2813237,X2813263,X2813289,X2813314,X2813378,X2813465,
0,chr1,16013,16014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,chr1,16533,16534,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...
2,chr1,16948,16949,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,chr1,17019,17020,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...
4,chr1,17745,17746,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [77]:
np.unique(GenomicFeatures_PCSI0023['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTN', 'CTT', 'GCA',
       'GCC', 'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'TCA', 'TCC',
       'TCG', 'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [78]:
PCSI0023 = pd.merge(GenomicFeatures_PCSI0023, SNVCalls_PCSI0023,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [79]:
PCSI0023.head() #182+15

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399007,X2701027,X2701030,
0,chr1,16014,138,47,5607,193,103,0,57.0,ACT,3.201550,0.656716,ncRNA_intronic,67070,0.523316,16013,16014,0,0,0,...
1,chr1,16534,69,46,3187,128,92,10,65.5,ACA,2.615385,0.442786,ncRNA_intronic,66550,0.078125,16533,16534,0,0,0,...
2,chr1,16949,194,34,6167,242,136,0,37.5,ATT,2.388889,0.597015,ncRNA_exonic,66135,0.921488,16948,16949,0,0,0,...
3,chr1,17020,249,44,8271,305,168,0,56.0,GCC,2.437500,0.587065,NaN,66064,0.881967,17019,17020,0,0,0,...
4,chr1,17746,326,55,11417,386,245,0,51.0,CTC,2.570423,0.587065,ncRNA_intronic,65338,0.274611,17745,17746,0,0,0,...


In [80]:
PCSI0023_X = PCSI0023.iloc[:,2:15] #Exclude CHROM, POS
PCSI0023_Y = PCSI0023.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [81]:
print PCSI0023_X.shape
print PCSI0023_Y.shape

(93177, 13)
(93177, 130)


In [82]:
PCSI0023_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,138,47,5607,193,103,0,57.0,ACT,3.201550,0.656716,ncRNA_intronic,67070,0.523316
1,69,46,3187,128,92,10,65.5,ACA,2.615385,0.442786,ncRNA_intronic,66550,0.078125


In [83]:
PCSI0023_Y[:2]

,X2399007,X2701027,X2701030,X2762077,X2762079,X2811068,X2811070,X2811073,X2811076,X2811391,X2812908,X2813237,X2813263,X2813289,X2813314,X2813378,X2813465,X2813649,X2813670,X2813677,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,...


In [84]:
totalSample = PCSI0023_Y.shape[0]
PCSI0023_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print PCSI0023_Y_Val.shape

93177
(93177, 1)


In [85]:
PCSI0023_Y.shape

(93177, 130)

In [86]:
PCSI0023_Y.values[0].shape

(130,)

In [87]:
for i in range(totalSample):
    PCSI0023_Y_Val[i] = np.argmax(np.bincount(PCSI0023_Y.values[i]))

In [88]:
PCSI0023_Y_Val_1 = pd.DataFrame(PCSI0023_Y_Val)
PCSI0023_Y_Val_1.columns = ['Label']

In [89]:
PCSI0023_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [90]:
PCSI0023_dataframe = pd.concat([PCSI0023_X, PCSI0023_Y_Val_1],axis=1)

In [91]:
PCSI0023_dataframe.shape

(93177, 14)

In [92]:
PCSI0023_dataframe[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,Label
0,138,47,5607,193,103,0,57.0,ACT,3.201550,0.656716,ncRNA_intronic,67070,0.523316,0
1,69,46,3187,128,92,10,65.5,ACA,2.615385,0.442786,ncRNA_intronic,66550,0.078125,0


In [35]:
PCSI0023_dataframe.to_csv('PCSI0023.csv')

In [36]:
#Now we have variable importance ...lets do analysis

In [37]:
%load_ext rpy2.ipython

In [39]:
%%R
# Read in the data
PCSI0023Frame = read.csv("PCSI0023.csv")
str(PCSI0023Frame)

'data.frame':	93177 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  138 69 194 249 326 87 127 69 105 37 ...
 $ NonRefAllele   : int  47 46 34 44 55 19 40 15 35 18 ...
 $ BaseQual       : int  5607 3187 6167 8271 11417 3291 4713 2477 4283 1234 ...
 $ TumourCoverage : int  193 128 242 305 386 117 168 84 143 56 ...
 $ NormalCoverage : int  103 92 136 168 245 89 118 69 100 31 ...
 $ MapQual        : num  0 10 0 0 0 0 20 0 37 29 ...
 $ ReadPosition   : num  57 65.5 37.5 56 51 50 55 45 42 27.5 ...
 $ Trinucleotide  : Factor w/ 33 levels "ACA","ACC","ACG",..: 4 1 8 19 14 13 11 32 13 20 ...
 $ HomopolymerRate: num  3.2 2.62 2.39 2.44 2.57 ...
 $ GCcontent      : num  0.657 0.443 0.597 0.587 0.587 ...
 $ GenomicLocation: Factor w/ 15 levels "","downstream",..: 8 8 7 1 8 8 5 5 5 1 ...
 $ DistanceSNP    : int  67070 66550 66135 66064 65338 54886 25792 18774 18471 921 ...
 $ StrandBias     : num  0.5233 0.0781 0.9215 0.882 0.2746 ...
 $ Label       

In [40]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
#sum(is.na(IS1Frame$ReadPosition)) 
sum(is.na(PCSI0023Frame)) 

[1] 0


In [33]:
#%%R
#CPCG0235Frame$ReadPosition[is.na(CPCG0235Frame$ReadPosition)] = mean(CPCG0235Frame$ReadPosition, na.rm=TRUE)
#CPCG0235Frame$StrandBias[is.na(CPCG0235Frame$StrandBias)] = mean(CPCG0235Frame$StrandBias, na.rm=TRUE)

#print (sum(is.na(CPCG0235Frame$ReadPosition)) )
#print (sum(is.na(CPCG0235Frame$StrandBias)) )

In [41]:
%%R
sum(is.na(PCSI0023Frame))  #No NA value now

[1] 0


In [42]:
%%R
write.csv(PCSI0023Frame, file = "PCSI0023_imputed.csv")  #Cleaned CSV without NA

In [43]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         49 minutes 32 seconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xqb196 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.58 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [44]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/PCSI0023_imputed.csv"
PCSI0023.hex          = h2o.importFile(path = csvPath, destination_frame = "PCSI0023.hex") 
train                 = as.data.frame(PCSI0023.hex)

str(train)

  |======================================================================| 100%
'data.frame':	93177 obs. of  16 variables:
 $ C1             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  138 69 194 249 326 87 127 69 105 37 ...
 $ NonRefAllele   : int  47 46 34 44 55 19 40 15 35 18 ...
 $ BaseQual       : int  5607 3187 6167 8271 11417 3291 4713 2477 4283 1234 ...
 $ TumourCoverage : int  193 128 242 305 386 117 168 84 143 56 ...
 $ NormalCoverage : int  103 92 136 168 245 89 118 69 100 31 ...
 $ MapQual        : num  0 10 0 0 0 0 20 0 37 29 ...
 $ ReadPosition   : num  57 65.5 37.5 56 51 50 55 45 42 27.5 ...
 $ Trinucleotide  : Factor w/ 33 levels "ACA","ACC","ACG",..: 4 1 8 19 14 13 11 32 13 20 ...
 $ HomopolymerRate: num  3.2 2.62 2.39 2.44 2.57 ...
 $ GCcontent      : num  0.657 0.443 0.597 0.587 0.587 ...
 $ GenomicLocation: Factor w/ 14 levels "downstream","exonic",..: 7 7 6 NA 7 7 4 4 4 NA ...
 $ DistanceSNP    : int  67

In [45]:
%%R

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%


In [46]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [47]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
                      variable relative_importance scaled_importance percentage
1                 NonRefAllele            1.000000          1.000000   0.264629
2                     BaseQual            0.398391          0.398391   0.105426
3               NormalCoverage            0.177425          0.177425   0.046952
4               TumourCoverage            0.156377          0.156377   0.041382
5 GenomicLocation.ncRNA_exonic            0.124313          0.124313   0.032897

---
                     variable relative_importance scaled_importance percentage
55          Trinucleotide.GTA            0.015645          0.015645   0.004140
56          Trinucleotide.ATT            0.012280          0.012280   0.003250
57   GenomicLocation.intronic            0.012165          0.012165   0.003219
58                 StrandBias            0.008272          0.008272   0.002189
59 GenomicLocation.intergenic            0.007674          0.007674   0.002031
60  Trinucleotide.

In [48]:
%%R
write.table(var_importance, file="PCSI0023_variable_importance.csv")

In [49]:
var_importance = pd.read_csv(open('PCSI0023_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [50]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.264629
2,BaseQual,0.398391,0.398391,0.105426
3,NormalCoverage,0.177425,0.177425,0.046952
4,TumourCoverage,0.156377,0.156377,0.041382
5,GenomicLocation.ncRNA_exonic,0.124313,0.124313,0.032897


In [51]:
GenomicFeatures_PCSI0023[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,16014,138,47,5607,193,103,0,57.0,ACT,3.201550,0.656716,ncRNA_intronic,67070,0.523316
1,chr1,16534,69,46,3187,128,92,10,65.5,ACA,2.615385,0.442786,ncRNA_intronic,66550,0.078125
2,chr1,16949,194,34,6167,242,136,0,37.5,ATT,2.388889,0.597015,ncRNA_exonic,66135,0.921488
3,chr1,17020,249,44,8271,305,168,0,56.0,GCC,2.437500,0.587065,NaN,66064,0.881967
4,chr1,17746,326,55,11417,386,245,0,51.0,CTC,2.570423,0.587065,ncRNA_intronic,65338,0.274611


In [52]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.39839103817939797,
 'DistanceSNP': 0.064937748014926897,
 'GCcontent': 0.023493539541959801,
 'GenomicLocation.UTR3': 0.026913395151495899,
 'GenomicLocation.UTR5': 0.052096251398324994,
 'GenomicLocation.downstream': 0.027410333976149601,
 'GenomicLocation.exonic': 0.046242468059062992,
 'GenomicLocation.exonic;splicing': 0.024703973904252101,
 'GenomicLocation.intergenic': 0.0076743708923459096,
 'GenomicLocation.intronic': 0.012164652347564701,
 'GenomicLocation.missing(NA)': 0.019427442923188199,
 'GenomicLocation.ncRNA_UTR3': 0.055960722267627702,
 'GenomicLocation.ncRNA_UTR5': 0.037618812173605,
 'GenomicLocation.ncRNA_exonic': 0.12431313842535001,
 'GenomicLocation.ncRNA_intronic': 0.029182266443967798,
 'GenomicLocation.splicing': 0.039804331958293901,
 'GenomicLocation.upstream': 0.044403795152902603,
 'GenomicLocation.upstream;downstream': 0.035438075661659199,
 'HomopolymerRate': 0.0792840421199799,
 'MapQual': 0.0979291126132011,
 'NonRefAllele': 1.0,
 'Norma

In [53]:
GenomicFeatures_PCSI0023.columns

Index([u'CHROM', u'POS', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'DistanceSNP', u'StrandBias'], dtype='object')

In [54]:
GenomicFeatures_PCSI0023.dtypes

CHROM               object
POS                  int64
RefAllele            int64
NonRefAllele         int64
BaseQual             int64
TumourCoverage       int64
NormalCoverage       int64
MapQual            float64
ReadPosition       float64
Trinucleotide       object
HomopolymerRate    float64
GCcontent          float64
GenomicLocation     object
DistanceSNP          int64
StrandBias         float64
dtype: object

In [55]:
GenomicFeatures_PCSI0023.iloc[0,13]

67070

In [56]:
GenomicFeatures_PCSI0023[:2]
#print GenomicFeatures_IS1.iloc[0,2]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,16014,138,47,5607,193,103,0,57.0,ACT,3.201550,0.656716,ncRNA_intronic,67070,0.523316
1,chr1,16534,69,46,3187,128,92,10,65.5,ACA,2.615385,0.442786,ncRNA_intronic,66550,0.078125


In [60]:
noOfSample = GenomicFeatures_PCSI0023.shape[0]

PCSI0023_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_PCSI0023.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_PCSI0023.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_PCSI0023.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_PCSI0023.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_PCSI0023.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_PCSI0023.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_PCSI0023.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_PCSI0023.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_PCSI0023.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_PCSI0023.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_PCSI0023.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_PCSI0023.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_PCSI0023.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_PCSI0023.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_PCSI0023.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_PCSI0023.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' Distance=',Distance
    #print 'StrandBias=',StrandBias
    PCSI0023_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [61]:
#Sorting dictionary
PCSI0023_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in PCSI0023_Dict.items()],reverse=True)]

In [62]:
PCSI0023_Dict_Frame = pd.DataFrame(PCSI0023_Dict1, columns=['POS', 'Value'])

In [63]:
PCSI0023_Dict_Frame.to_csv("PCSI0023_Dict_Frame.csv")

In [93]:
PCSI0023_Dict_Frame = pd.read_csv("PCSI0023_Dict_Frame.csv")

In [94]:
PCSI0023_Dict_Frame[:10]

,Unnamed: 0,POS,Value
0,0,24488953,216534.588791
1,1,24488937,216533.505030
2,2,78192,170722.594353
3,3,78172,170678.086136
4,4,78732,170532.353723
5,5,159757,166204.947858
6,6,156853,165858.293017
7,7,164233,165393.430830
8,8,185097,164995.691528
9,9,171226,164895.786137


In [95]:
GenomicFeatures_PCSI0023_1 = pd.concat([PCSI0023.iloc[:,0:16], PCSI0023_Y_Val_1],axis=1)

In [96]:
rows = PCSI0023_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_PCSI0023_Top20 = GenomicFeatures_PCSI0023_1.loc[GenomicFeatures_PCSI0023_1['POS_x'].isin(rows)]

[24488953 24488937    78192    78172    78732   159757   156853   164233
   185097   171226   176368   185537   223155   227321 42385188   368526
   380702   426897   458998   461518]


In [98]:
GenomicFeatures_PCSI0023_Top20[:10]

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,Label
8327,chr10,42385188,6569,1429,247986,181542,170360,23,40.0,CTT,2.171233,0.368159,NaN,12185,0.472249,42385187,0
89070,chrX,78172,40,3,1081,44,29,0,50.5,CCG,1.856250,0.492537,intergenic,2621383,0.500000,78171,0
89071,chrX,78192,40,0,1204,41,31,0,49.0,TTG,1.881250,0.492537,intergenic,2621363,0.439024,78191,0
89072,chrX,78732,28,9,797,42,25,0,46.5,CCG,1.911392,0.517413,NaN,2620823,0.428571,78731,0
89073,chrX,156853,54,5,1791,66,28,0,56.5,CTC,2.006410,0.492537,intergenic,2542702,0.560606,156852,0
89074,chrX,159757,100,0,3087,134,85,0,36.5,CTA,1.797546,0.507463,intergenic,2539798,0.537313,159756,0
89075,chrX,164233,68,4,1816,73,45,0,59.0,ACC,1.771605,0.512438,intergenic,2535322,0.630137,164232,0
89076,chrX,171226,47,9,1703,58,51,0,48.0,CTC,2.214765,0.572139,intergenic,2528329,0.413793,171225,0
89077,chrX,176368,10,2,240,12,10,0,30.5,GTA,2.993377,0.577114,NaN,2523187,1.000000,176367,0
89078,chrX,185097,138,4,4157,149,82,0,55.0,ATA,3.621849,0.626866,intergenic,2514458,0.771812,185096,0


In [67]:
GenomicFeatures_PCSI0023_Top20.to_csv('GenomicFeatures_IS4_Top20.csv')